<a href="https://colab.research.google.com/github/Pt-home/Colab_Northwind_SQL_Learning/blob/main/Northwind_SQL_Colab_Ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧠 Навчальний SQL-блокнот з базою Northwind

Цей Jupyter Notebook створено для практики SQL-запитів на прикладі класичної навчальної бази **Northwind**.

- 📚 Теорія SQL: [W3Schools SQL Tutorial](https://www.w3schools.com/sql/)



## 📘 Структура бази даних Northwind

У базі є такі основні таблиці:

- **Customers** — Клієнти
- **Orders** — Замовлення
- **OrderDetails** — Деталі замовлень
- **Products** — Продукти
- **Suppliers** — Постачальники
- **Categories** — Категорії
- **Employees** — Співробітники
- **Shippers** — Перевізники

Схема відображає типову бізнес-логіку: замовлення клієнтів на продукти, постачання, доставка, працівники.


In [12]:

import sqlite3
import pandas as pd
import os
import requests


In [13]:
#url = "https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/main/src/create.sql"
#basename = "create.sql"

url = "https://github.com/Pt-home/Colab_Northwind_SQL_Learning/raw/refs/heads/main/Northwind.sql"
basename = "Northwind.sql"

In [14]:

db_name = "MyNorthwind.sqlite"

if os.path.exists(db_name):
    conn = sqlite3.connect(db_name)
    print("📂 Завантажено збережену базу MyNorthwind.sqlite")
else:
    print("⬇️ Завантаження та створення бази")
    r = requests.get(url)
    with open(basename, "wb") as f:
        f.write(r.content)

    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    with open(basename, "r", encoding="utf-8") as f:
        sql_script = f.read()
    cursor.executescript(sql_script)
    conn.commit()
    print(f"✅ Базу створено з {basename}")

cursor = conn.cursor()


⬇️ Завантаження та створення бази
✅ Базу створено з Northwind.sql


In [15]:
def describe_all_tables():

    conn = sqlite3.connect("MyNorthwind.sqlite")  # або інша назва файлу
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'")
    tables = [row[0] for row in cursor.fetchall()]

    for table in tables:
        print(f"\n📄 Структура таблиці: {table}")
        # Екранування назви таблиці в лапки
        query = f'PRAGMA table_info("{table}");'
        df = pd.read_sql_query(query, conn)
        display(df)

describe_all_tables()


📄 Структура таблиці: Customers


,cid,name,type,notnull,dflt_value,pk
0,0,CustomerID,INTEGER,0,None,1
1,1,CustomerName,TEXT,0,None,0
2,2,ContactName,TEXT,0,None,0
3,3,Address,TEXT,0,None,0
4,4,City,TEXT,0,None,0
5,5,PostalCode,TEXT,0,None,0
6,6,Country,TEXT,0,None,0



📄 Структура таблиці: Categories


,cid,name,type,notnull,dflt_value,pk
0,0,CategoryID,INTEGER,0,None,1
1,1,CategoryName,TEXT,0,None,0
2,2,Description,TEXT,0,None,0



📄 Структура таблиці: Shippers


...

,cid,name,type,notnull,dflt_value,pk
0,0,OrderID,INTEGER,0,None,1
1,1,CustomerID,INTEGER,0,None,0
2,2,EmployeeID,INTEGER,0,None,0
3,3,OrderDate,TEXT,0,None,0
4,4,ShipperID,INTEGER,0,None,0



📄 Структура таблиці: Employees


,cid,name,type,notnull,dflt_value,pk
0,0,EmployeeID,INTEGER,0,None,1
1,1,LastName,TEXT,0,None,0
2,2,FirstName,TEXT,0,None,0
3,3,BirthDate,TEXT,0,None,0
4,4,Photo,TEXT,0,None,0
5,5,Notes,TEXT,0,None,0



📄 Структура таблиці: Products


,cid,name,type,notnull,dflt_value,pk
0,0,ProductID,INTEGER,0,None,1
1,1,ProductName,TEXT,0,None,0
2,2,SupplierID,INTEGER,0,None,0
3,3,CategoryID,INTEGER,0,None,0
4,4,Unit,TEXT,0,None,0
5,5,Price,REAL,0,None,0


In [16]:
def describe_table(table_name):
    df = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
    display(df)

describe_table("Customers")

,cid,name,type,notnull,dflt_value,pk
0,0,CustomerID,INTEGER,0,None,1
1,1,CustomerName,TEXT,0,None,0
2,2,ContactName,TEXT,0,None,0
3,3,Address,TEXT,0,None,0
4,4,City,TEXT,0,None,0
5,5,PostalCode,TEXT,0,None,0
6,6,Country,TEXT,0,None,0


In [18]:

def run_sql(query):
    try:
        if query.strip().lower().startswith("select"):
            df = pd.read_sql_query(query, conn)
            display(df)
            return df
        else:
            cursor.execute(query)
            conn.commit()
            print("✅ Запит виконано та зміни збережено")
    except Exception as e:
        print("❌ Помилка:", e)


In [19]:

query = input("Введіть SQL-запит: ")
result = run_sql(query)


Введіть SQL-запит: SELECT CustomerName, City, Country FROM Customers


,CustomerName,City,Country
0,Alfreds Futterkiste,Berlin,Germany
1,Ana Trujillo Emparedados y helados,México D.F.,Mexico
2,Antonio Moreno Taquería,México D.F.,Mexico
3,Around the Horn,London,UK
4,Berglunds snabbköp,Luleå,Sweden
...,...,...,...
86,Wartian Herkku,Oulu,Finland
87,Wellington Importadora,Resende,Brazil
88,White Clover Markets,Seattle,USA
89,Wilman Kala,Helsinki,Finland


In [ ]:

if 'result' in locals() and result is not None:
    result.to_csv("sql_result.csv", index=False)
    print("💾 Збережено в sql_result.csv")


In [ ]:

backup_file = "MyNorthwind.sqlite"
conn.backup(sqlite3.connect(backup_file))
print(f"📦 Копія бази збережена як {backup_file}")
